In [1]:
import re
from transformers import StoppingCriteria, StoppingCriteriaList
from dataset import get_MATH_dataset
from utils import get_pipe
from evaluation import evaluate

model_path = "models/checkpoint-1200"
completion_prompt = "{problem} \nPlease reason step by step, and put your final answer within \\boxed{{}}.\nApproach: "

dataset = get_MATH_dataset("test")
pipe = get_pipe(model_path)


class StoppingCriteriaSub(StoppingCriteria):
    def __call__(self, input_ids, scores):
        decoded_text = pipe.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return re.search(r"\\boxed\{(.+)\}", decoded_text) is not None


generate_kargs = {
    "max_new_tokens": 1000,
    "do_sample": True,
    "batch_size": 32,
    "top_k": 0.0,
    "top_p": 0.85,
    "temperature": 0.85,
    "stopping_criteria": StoppingCriteriaList([StoppingCriteriaSub()]),
}


def get_answer_from_output(text):
    try:
        result_output = re.findall(r"\\boxed\{(\d+)\}", text)
        return float(result_output[0])
    except Exception:
        return None


accuracy, history = evaluate(
    pipe=pipe,
    dataset=dataset,
    prompt=completion_prompt,
    generate_kargs=generate_kargs,
    return_history=True,
    extract_answer=get_answer_from_output,
    batch_size=32,
)

Resolving data files:   0%|          | 0/7500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5000 [00:00<?, ?it/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/ntuai/AIMO/.venv/lib/python3.10/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Evaluating accuracy. Current accuracy: 0.00:   0%|          | 0/86 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
